# Assignment 7

In [1]:
# Set environment params
Sys.setenv(LANG='en')  # English

# Import libraries
library(mlr)           # ML toolkit
library(caret)         # ML toolkit
library(nnet)          # class.ind() function
library(neuralnet)     # Deep Neural Networks
library(LiblineaR)     # LR Lasso (l1)
library(randomForest)  # Random Forest
library(adabag)        # Boosting
library(e1071)         # SVM
library(ggplot2)       # Visualization
library(plotly)        # 3D visualization

# Import data
library(ISLR)      # Data from the course book
library(MASS)      # Boston housing dataset
library(datasets)  # US crime dataset

# Resize plot
library(repr)  # String and binary representations

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: ggplot2

Loading required package: lattice


Attaching package: 'caret'


The following object is masked from 'package:mlr':

    train


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


Loading required package: rpart

Loading required package: foreach

Loading required package: doParallel

Loading required package: iterators

Loading required package: parallel


Attaching package: 'e1071'


The following object is masked from 'package:mlr':

    impute



Attaching package: 'plotly'


The following object is masked from 'package:ggplot2':

   

In [2]:
data_df <- read.csv('./data/bankruptcy_prediction/data.csv')

In [4]:
head(data_df)

,Bankrupt.,ROA.C..before.interest.and.depreciation.before.interest,ROA.A..before.interest.and...after.tax,ROA.B..before.interest.and.depreciation.after.tax,Operating.Gross.Margin,Realized.Sales.Gross.Margin,Operating.Profit.Rate,Pre.tax.net.Interest.Rate,After.tax.net.Interest.Rate,Non.industry.income.and.expenditure.revenue,⋯,Net.Income.to.Total.Assets,Total.assets.to.GNP.price,No.credit.Interval,Gross.Profit.to.Sales,Net.Income.to.Stockholder.s.Equity,Liability.to.Equity,Degree.of.Financial.Leverage..DFL.,Interest.Coverage.Ratio..Interest.expense.to.EBIT.,Net.Income.Flag,Equity.to.Liability
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,1,0.3705943,0.4243894,0.4057498,0.6014572,0.6014572,0.9989692,0.7968871,0.8088094,0.3026464,⋯,0.7168453,0.009219440,0.6228790,0.6014533,0.8278902,0.2902019,0.02660063,0.5640501,1,0.01646874
2,1,0.4642909,0.5382141,0.5167300,0.6102351,0.6102351,0.9989460,0.7973802,0.8093007,0.3035564,⋯,0.7952971,0.008323302,0.6236517,0.6102365,0.8399693,0.2838460,0.26457682,0.5701749,1,0.02079431
3,1,0.4260713,0.4990188,0.4722951,0.6014500,0.6013635,0.9988574,0.7964034,0.8083875,0.3020352,⋯,0.7746697,0.040002853,0.6238410,0.6014493,0.8367743,0.2901885,0.02655472,0.5637061,1,0.01647411
4,1,0.3998440,0.4512647,0.4577333,0.5835411,0.5835411,0.9986997,0.7969670,0.8089656,0.3033495,⋯,0.7395545,0.003252475,0.6229287,0.5835376,0.8346971,0.2817212,0.02669663,0.5646634,1,0.02398233
5,1,0.4650222,0.5384322,0.5222978,0.5987835,0.5987835,0.9989731,0.7973661,0.8093037,0.3034750,⋯,0.7950159,0.003877563,0.6235207,0.5987815,0.8399727,0.2785138,0.02475185,0.5756166,1,0.03549020
6,1,0.3886803,0.4151766,0.4191338,0.5901714,0.5902507,0.9987581,0.7969032,0.8087706,0.3031158,⋯,0.7104205,0.005277875,0.6226046,0.5901723,0.8299390,0.2850871,0.02667537,0.5645383,1,0.01953448


In [7]:
sum(is.na(data_df))

[1] 0

In [12]:
unique(data_df$Bankrupt)

[1] 1 0

In [13]:
length(data_df)

[1] 96

In [11]:
## 80% of the sample size
smp_size <- floor(0.80 * nrow(data_df))

## set the seed to make your partition reproducible
set.seed(1)
train_ind <- sample(seq_len(nrow(data_df)), size = smp_size)

train <- data_df[train_ind, ]
test <- data_df[-train_ind, ]

In [19]:
md_nnet <- neuralnet(Bankrupt.~.,
                     train,
                     hidden=c(1, 30),      # Size of the hidden layers
                     #threshold=0.1,          # Stopping criteria, a.k.a convergence
                     stepmax=5000,            # Maximum training step
                     rep=10,                  # Number of training repeat, a.k.a epoch
                     lifesign='full',         # Print training process
                     lifesign.step=5000,      # Print out every 5000 steps
                     algorithm='rprop+',      # Algorithm to calculate the network, 'rprop+'=resilient backpropagation
                     learningrate=0.01,       # Learning rate, only use for traditional backpropagation
                     err.fct='sse',           # Error function, sse=sum square error, ce=cross-entropy
                     act.fct="logistic",      # Activation function, 'logistic' or 'tanh'
                     linear.output=F
                    )

hidden: 1, 30    thresh: 0.01    rep:  1/10    steps: 
     25
	error: 85.15136
	time: 0.95 secs

hidden: 1, 30    thresh: 0.01    rep:  2/10    steps: 
     27
	error: 85.16077
	time: 0.52 secs

hidden: 1, 30    thresh: 0.01    rep:  3/10    steps: 
     31
	error: 85.16677
	time: 0.78 secs

hidden: 1, 30    thresh: 0.01    rep:  4/10    steps: 
     33
	error: 85.15357
	time: 0.74 secs

hidden: 1, 30    thresh: 0.01    rep:  5/10    steps: 
     32
	error: 85.16183
	time: 0.52 secs

hidden: 1, 30    thresh: 0.01    rep:  6/10    steps: 
     23
	error: 85.16299
	time: 0.4 secs

hidden: 1, 30    thresh: 0.01    rep:  7/10    steps: 
     48
	error: 85.13325
	time: 0.9 secs

hidden: 1, 30    thresh: 0.01    rep:  8/10    steps: 
     55
	error: 85.16073
	time: 1.1 secs

hidden: 1, 30    thresh: 0.01    rep:  9/10    steps: 
     31
	error: 85.16077
	time: 0.77 secs

hidden: 1, 30    thresh: 0.01    rep: 10/10    steps: 
     38
	error: 85.1673 
	time: 0.72 secs



In [20]:
# Make prediction and evaluation on train
y_train_pred <- predict(md_nnet, train[, c(2:96)])
mean((max.col(y_train_pred)-1) == train$Bankrupt)

[1] 0.967736

In [21]:
# Make prediction and evaluation on test
y_test_pred <- predict(md_nnet, test[, c(2:96)])
mean((max.col(y_test_pred)-1) == test$Bankrupt)

[1] 0.9677419

In [22]:
md_nnet <- neuralnet(Bankrupt.~.,
                     train,
                     hidden=c(30, 30),      # Size of the hidden layers
                     #threshold=0.1,          # Stopping criteria, a.k.a convergence
                     stepmax=5000,            # Maximum training step
                     rep=10,                  # Number of training repeat, a.k.a epoch
                     lifesign='full',         # Print training process
                     lifesign.step=5000,      # Print out every 5000 steps
                     algorithm='rprop+',      # Algorithm to calculate the network, 'rprop+'=resilient backpropagation
                     learningrate=0.01,       # Learning rate, only use for traditional backpropagation
                     err.fct='sse',           # Error function, sse=sum square error, ce=cross-entropy
                     act.fct="logistic",      # Activation function, 'logistic' or 'tanh'
                     linear.output=F
                    )

hidden: 30, 30    thresh: 0.01    rep:  1/10    steps: 
   2514
	error: 80.51917
	time: 1.02 mins

hidden: 30, 30    thresh: 0.01    rep:  2/10    steps: 
   1179
	error: 82.93225
	time: 27.47 secs

hidden: 30, 30    thresh: 0.01    rep:  3/10    steps: 
      3
	error: 87.99077
	time: 0.09 secs

hidden: 30, 30    thresh: 0.01    rep:  4/10    steps: 
   1637
	error: 83.2496 
	time: 37.29 secs

hidden: 30, 30    thresh: 0.01    rep:  5/10    steps: 
   2381
	error: 81.7835 
	time: 53.99 secs

hidden: 30, 30    thresh: 0.01    rep:  6/10    steps: 
   2152
	error: 85.26468
	time: 50.43 secs

hidden: 30, 30    thresh: 0.01    rep:  7/10    steps: 
   2486
	error: 81.80581
	time: 56.6 secs

hidden: 30, 30    thresh: 0.01    rep:  8/10    steps: 
   1823
	error: 80.35597
	time: 41.8 secs

hidden: 30, 30    thresh: 0.01    rep:  9/10    steps: 
   1287
	error: 82.05561
	time: 32.53 secs

hidden: 30, 30    thresh: 0.01    rep: 10/10    steps: 
   2011
	error: 82.43451
	time: 45.52 secs



In [23]:
# Make prediction and evaluation on train
y_train_pred <- predict(md_nnet, train[, c(2:96)])
mean((max.col(y_train_pred)-1) == train$Bankrupt)

[1] 0.967736

In [24]:
# Make prediction and evaluation on test
y_test_pred <- predict(md_nnet, test[, c(2:96)])
mean((max.col(y_test_pred)-1) == test$Bankrupt)

[1] 0.9677419

In [25]:
# %*% dot product, * element wise product
nnet <- function(X, Y, step_size=0.5, reg=0.001, h=10, niteration) {
    "
    This function construct a simple 1-hidden layer neural network with ReLU activation function.
    "
  
    # Get dim of input
    N <- nrow(X)  # Number of examples
    K <- ncol(Y)  # Number of classes
    D <- ncol(X)  # Dimensionality

    # Initialize parameters randomly
    # Hidden layer
    W <- 0.01 * matrix(rnorm(D * h), nrow=D)
    b <- matrix(0, nrow=1, ncol=h)
    # Output layer
    W2 <- 0.01 * matrix(rnorm(h * K), nrow=h)
    b2 <- matrix(0, nrow=1, ncol=K)

    # Gradient descent loop to update weight and bias
    for (i in 0:niteration) {
    
        #--------------------------------------------
        # Forward propagation
        #--------------------------------------------
        
        # Hidden layer, ReLU activation
        hidden_layer <- pmax(0, X %*% W + matrix(rep(b, N), nrow=N, byrow=T))  # ReLU
        hidden_layer <- matrix(hidden_layer, nrow=N)
        
        # Class score
        scores <- hidden_layer %*% W2 + matrix(rep(b2, N), nrow=N, byrow=T)

        # Compute and normalize class probabilities
        exp_scores <- exp(scores)
        probs <- exp_scores / rowSums(exp_scores)
       
        # Compute the loss: sofmax and regularization
        corect_logprobs <- -log(probs)
        data_loss <- sum(corect_logprobs * Y) / N
        reg_loss <- 0.5 * reg * sum(W * W) + 0.5 * reg * sum(W2 * W2)
        loss <- data_loss + reg_loss
        
        # Check progress
        if (i %% 1000 == 0 | i == niteration) {
          print(paste("iteration", i, ': loss', loss))
        }
        
        #--------------------------------------------
        # Backward propagation
        #--------------------------------------------

        # Compute the gradient on scores
        dscores <- probs - Y
        dscores <- dscores / N

        # Backpropate the gradient to the parameters
        dW2 <- t(hidden_layer) %*% dscores
        db2 <- colSums(dscores)
        
        # Next backprop into hidden layer
        dhidden <- dscores %*% t(W2)
        
        # Backprop the ReLU non-linearity
        dhidden[hidden_layer <= 0] <- 0
        
        # Finally into W, b
        dW <- t(X) %*% dhidden
        db <- colSums(dhidden)

        # Add regularization gradient contribution
        dW2 <- dW2 + reg * W2
        dW <- dW + reg * W

        # Update parameter 
        W <- W - step_size * dW
        b <- b - step_size * db
        W2 <- W2 - step_size * dW2
        b2 <- b2 - step_size * db2
    }
    
    return(list(W, b, W2, b2))
}

# Function to make prediction using Neural Networks
nnetPred <- function(X, para=list()) {
    
    # The trained params of NN
    W <- para[[1]]
    b <- para[[2]]
    W2 <- para[[3]]
    b2 <- para[[4]]

    N <- nrow(X)
    hidden_layer <- pmax(0, X %*% W + matrix(rep(b, N), nrow=N, byrow=T)) 
    hidden_layer <- matrix(hidden_layer, nrow=N)
    scores <- hidden_layer %*% W2 + matrix(rep(b2, N), nrow=N, byrow=T)  # Linear output
    predicted_class <- apply(scores, 1, which.max)

    return(predicted_class)  
}

In [26]:
# Preprocessing train, test data
nzv <- nearZeroVar(train)  # Drop zero-variance variables
nzv.nolabel <- nzv - 1

# Train data
X_train <- as.matrix(train[, -1])  # Data matrix (each row = single example)
N <- nrow(X_train)  # Number of examples
y_train <- train[, 1]  # Class labels

K <- length(unique(y_train))  # Number of classes
X_train_proccessed <- X_train[, -nzv.nolabel] / max(X_train)  # Scale train data
D <- ncol(X_train_proccessed)  # Dimensionality

# Test data
X_test <- as.matrix(test[, -1])  # Data matrix (each row = single example)
y_test <- test[, 1]  # Class labels
X_test_proccessed <- X_test[, -nzv.nolabel] / max(X_train)  # Scale test data using train info

# Dummitize the y_train
y_train_dummy <- matrix(0, N, K)
for (i in 1:N) {
    y_train_dummy[i, y_train[i] + 1] <- 1
}

In [27]:
#Run the Neural Networks function
nnet.mnist <- nnet(X_train_proccessed, y_train_dummy, niteration=7000)

[1] "iteration 0 : loss 0.693094461143782"
[1] "iteration 1000 : loss 0.139987375150091"
[1] "iteration 2000 : loss 0.139103907104233"
[1] "iteration 3000 : loss 0.138967649784129"
[1] "iteration 4000 : loss 0.138882681711823"
[1] "iteration 5000 : loss 0.138782624769643"
[1] "iteration 6000 : loss 0.138745078671579"
[1] "iteration 7000 : loss 0.138726304052672"


In [28]:
# Make prediction on train set
predicted_class <- nnetPred(X_train_proccessed, nnet.mnist)
print(paste('Train accuracy:', mean(predicted_class == (y_train+1))))

[1] "Train accuracy: 0.967736021998167"


In [29]:
# Make prediction on test set
predicted_class <- nnetPred(X_test_proccessed, nnet.mnist)
print(paste('Test accuracy:', mean(predicted_class == (y_test+1))))

[1] "Test accuracy: 0.967741935483871"


In [33]:
# Define the ML classification task
train_task <- mlr::makeClassifTask(id ='MNIST_train', data=train, target='Bankrupt.')
test_task <- mlr::makeClassifTask(id='MNIST_test', data=test, target='Bankrupt.')

In [34]:
# Logistic Regression Lasso (l1)
learner <- mlr::makeLearner('classif.LiblineaRL1LogReg')  # Register a machine learning model
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task, proba=T)
performance(pred_test, measures=acc)

acc 
0.9706745

In [35]:
# k-Nearest Neighbor (k=50)
learner <- makeLearner('classif.knn', k=50)
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.9677419

In [36]:
# LDA (drop zero-variance features)
learner <- makeLearner('classif.lda')
model <- mlr::train(learner, filterFeatures(train_task, method='variance', threshold=0.1))
#model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.9582111

In [37]:
# Decision Tree
learner <- mlr::makeLearner('classif.rpart')  # Register a machine learning model
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.9655425

In [38]:
# Random Forest
learner <- makeLearner('classif.randomForest')
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.9684751

In [39]:
# Adabag Boosting
learner <- makeLearner('classif.boosting')
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.9721408

In [40]:
# SVM
learner <- makeLearner('classif.svm', scale=FALSE, kernel='linear')  # linear,polynomial,radial,sigmoid
model <- mlr::train(learner, train_task)
pred_test <- predict(model, task=test_task)
performance(pred_test, measures=acc)

acc 
0.5417889